In [ ]:
import keras
from tensorflow.keras.applications import ResNet50V2
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/Signs.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

dataGenerator = keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3, samplewise_center = True)

test = dataGenerator.flow_from_directory("/tmp/Signs", class_mode='categorical', target_size = (64,64), batch_size=30, subset="validation", shuffle=True)
train = dataGenerator.flow_from_directory("/tmp/Signs", class_mode='categorical', target_size = (64,64), batch_size=30, subset="training", shuffle=True)

Found 4917 images belonging to 2 classes.
Found 11479 images belonging to 2 classes.


In [ ]:
def stopSignNet():
  
    model = keras.models.Sequential()
    #model.add(keras.layers.experimental.preprocessing.Normalization(input_shape = (64,64,3)))
    model.add(keras.layers.experimental.preprocessing.RandomZoom(0.2, input_shape = (64,64,3)))
    model.add(keras.layers.experimental.preprocessing.RandomTranslation(height_factor = 0.1, width_factor = 0.1))
    model.add(ResNet50V2(include_top = False, weights = "imagenet", input_tensor = keras.Input(shape = (64, 64, 3))))
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(2, activation = "softmax"))

    #iterationsToChangeAt = [60,120]
    #LRs = [.001, .0001, 0.00001]
    #stepRate = keras.optimizers.schedules.PiecewiseConstantDecay(iterationsToChangeAt, LRs)

    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate = 0.001), metrics=["categorical_accuracy"])
    
    return(model)

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-4, mode='min')
    
model = stopSignNet()
model.summary()
#model.fit(train, epochs=14, validation_data=test, class_weight = {0:1, 1:8}, callbacks=[mcp_save, reduce_lr_loss])
model.fit(train, epochs=100, validation_data=test, callbacks=[mcp_save, reduce_lr_loss])

#IMPORTANT: Your model must be saved as "Q3.h5"
model.save("./Q1.h5")

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_zoom_7 (RandomZoom)   (None, 64, 64, 3)         0         
_________________________________________________________________
random_translation_7 (Random (None, 64, 64, 3)         0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2, 2, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_7 ( (None, 2048)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 4098      
Total params: 23,568,898
Trainable params: 23,523,458
Non-trainable params: 45,440
_____________________________________